Data 불러와서 나누기

In [203]:
import pickle
with open('./data/train_token.pickle', 'rb') as fs:
    train_token = pickle.load(fs)

with open('./data/dev_token.pickle', 'rb') as fs:
    dev_token = pickle.load(fs)

Data 탐색

데이터 복원 & 형태소 분석과 pos tagging

In [313]:
# from eunjeon import Mecab
# mecab = Mecab()
from konlpy.tag import Okt
okt = Okt()

doPos = lambda x: okt.pos(x)
train_sentence = list(map(' '.join, train_token))
dev_sentence = list(map(' '.join, dev_token))

# Pos tagging
pos_train = list(map(doPos, train_sentence))
pos_dev = list(map(doPos, dev_sentence))

# Token 만들기 
getPos = lambda x: [pos for morph, pos in x]
train_pos = list(map(getPos, pos_train))
train_dev = list(map(getPos, pos_dev))

In [282]:
with open('pos_train.pickle', 'wb') as fs:
    pickle.dump(pos_train, fs)
with open('pos_dev.pickle', 'wb') as fs:
    pickle.dump(pos_dev, fs)

B / I tagging 하기

pos_vocab 만들기

In [377]:
import pickle 
with open('./data/train_tag.pickle', 'rb') as fs:
    train_tag = pickle.load(fs)
with open('./data/dev_tag.pickle', 'rb') as fs:
    dev_tag = pickle.load(fs)

In [397]:
pos_vocab = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation', 'Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation', 'Foreign', 'Alpha', 'Number', 'Unknown', 'KoreanParticle', 'Hashtag', 'ScreenName', 'Email', 'URL']

# train, dev 데이터에서 품사 집합 구해서 pos_vocab에 추가
pos_set_train = set([y for x in train_pos for y in x])
pos_set_dev = set([y for x in train_dev for y in x])

pos_set = list(pos_set_train | pos_set_dev)

pos_vocab = sorted(list(set(pos_vocab) | set(pos_set)))

One_hot 벡터 만들기

In [379]:
import numpy as np

def posOnehot(sentence):
    onehot_list = []
    for word in sentence:
        onehot = np.zeros(len(pos_vocab))
        ind = pos_vocab.index(word)
        onehot[ind] += 1 
            
        onehot_list.append(onehot)
    return onehot_list

train_onehot = list(map(posOnehot, train_pos))
dev_onehot = list(map(posOnehot, train_dev))

In [383]:
def makePosDict(token, key_list, onehot_vector):
    output = {}
    for i, sentence in enumerate(token):
        for j, word in enumerate(sentence):
            key = key_list[key_list.index(word)]
            output[key] = onehot_vector[i][j]
    return output

key_list_train = list(set([y for x in train_token for y in x]))
key_list_dev = list(set([y for x in dev_token for y in x]))

train_dict = makePosDict(train_token, key_list_train, train_onehot)
dev_dict = makePosDict(dev_token, key_list_dev, dev_onehot) 

In [392]:
with open('train_dict.pickle', 'wb') as fs:
    pickle.dump(train_dict, fs)
with open('dev_dict.pickle', 'wb') as fs:
    pickle.dump(dev_dict, fs)